## Импорт библиотек

In [ ]:
import re
import logging
import numpy as np
import sympy as sp
from abc import ABC, abstractmethod
from dataclasses import dataclass
from time import perf_counter

## Интерфейс для задач оптимизации

In [ ]:
class NLPProblemInterface(ABC):
    @property
    @abstractmethod
    def n(self):
        pass
    
    @property
    @abstractmethod
    def m(self):
        pass
    
    @property
    @abstractmethod
    def cl(self):
        pass
    
    @property
    @abstractmethod
    def cu(self):
        pass
    
    @property
    @abstractmethod
    def x0(self):
        pass
    
    @abstractmethod
    def obj(self, x):
        pass
    
    @abstractmethod
    def grad(self, x):
        pass
    
    @abstractmethod
    def cons(self, x):
        pass
    
    @abstractmethod
    def jac(self, x):
        pass
    
    @abstractmethod
    def hess(self, x, y):
        pass

## Реализация интерфейса для pycutest задач

In [ ]:
class PyCUTestProblem(NLPProblemInterface):
    def __init__(self, cutest_problem, mu0=None, epsilon=None):
        self._prob = cutest_problem
        self._n = cutest_problem.n
        self._m = cutest_problem.m
        self._x0 = cutest_problem.x0.copy()
        
        if self._m > 0:
            self._cl = cutest_problem.cl.copy()
            self._cu = cutest_problem.cu.copy()
        else:
            self._cl = np.array([])
            self._cu = np.array([])
        
        self.mu0 = mu0
        self.epsilon = epsilon
    
    @property
    def n(self):
        return self._n
    
    @property
    def m(self):
        return self._m
    
    @property
    def cl(self):
        return self._cl
    
    @property
    def cu(self):
        return self._cu
    
    @property
    def x0(self):
        return self._x0
    
    def obj(self, x):
        return float(self._prob.obj(x))
    
    def grad(self, x):
        _, g = self._prob.obj(x, gradient=True)
        return np.asarray(g, dtype=float)
    
    def cons(self, x):
        if self._m == 0:
            return np.zeros(0, dtype=float)
        c = self._prob.cons(x)
        return np.asarray(c, dtype=float)
    
    def jac(self, x):
        if self._m == 0:
            return np.zeros((0, self._n), dtype=float)
        _, J = self._prob.cons(x, gradient=True)
        return np.asarray(J, dtype=float)
    
    def hess(self, x, y):
        if y.size != self._m:
            raise ValueError(f"hess: y has size {y.size} but m={self._m}")
        H = self._prob.hess(x, y)
        return np.asarray(H, dtype=float)

## Реализация интерфейса для txt-файлов

In [ ]:
class TxtNLPProblem(NLPProblemInterface):
    def __init__(self, var_names, f_expr, c_exprs, cl, cu, x0,
                 s0=None, lam0=None, nu0=None, mu0=None, epsilon=None):
        self.var_names = list(var_names)
        self._n = len(self.var_names)

        self._x_syms = sp.symbols(self.var_names, real=True)
        x_syms = self._x_syms

        self._f_expr = f_expr
        self._f_func = sp.lambdify(x_syms, f_expr, "numpy")
        grad_expr = [sp.diff(f_expr, xi) for xi in x_syms]
        self._grad_func = sp.lambdify(x_syms, grad_expr, "numpy")
        self._Hf_expr = sp.hessian(f_expr, x_syms)

        self._c_exprs = list(c_exprs)
        self._m = len(self._c_exprs)
        self._cl = np.asarray(cl, dtype=float)
        self._cu = np.asarray(cu, dtype=float)

        self._c_func = sp.lambdify(x_syms, self._c_exprs, "numpy")
        J_expr = sp.Matrix(self._c_exprs).jacobian(x_syms)
        self._J_func = sp.lambdify(x_syms, J_expr, "numpy")

        self._Hc_exprs = [sp.hessian(ci, x_syms) for ci in self._c_exprs]

        self._Hf_func = sp.lambdify(x_syms, self._Hf_expr, "numpy")
        self._Hc_funcs = [
            sp.lambdify(x_syms, Hci, "numpy")
            for Hci in self._Hc_exprs
        ]

        self._x0 = np.asarray(x0, dtype=float)

        self.s0 = None if s0 is None else np.asarray(s0, dtype=float)
        self.lam0 = None if lam0 is None else np.asarray(lam0, dtype=float)
        self.nu0 = None if nu0 is None else np.asarray(nu0, dtype=float)
        self.mu0 = None if mu0 is None else float(mu0)
        self.epsilon = None if epsilon is None else float(epsilon)

    @property
    def n(self):
        return self._n
    
    @property
    def m(self):
        return self._m
    
    @property
    def cl(self):
        return self._cl
    
    @property
    def cu(self):
        return self._cu
    
    @property
    def x0(self):
        return self._x0

    @staticmethod
    def _parse_keyvals(line: str):
        key, val = line.split(":", 1)
        return key.strip(), val.strip()

    @staticmethod
    def _find_vars_in_exprs(expr_strs):
        names = set()
        for s in expr_strs:
            for m in re.findall(r"\bx\d+\b", s):
                names.add(m)

        def keyfn(v):
            return int(v[1:])
        return sorted(names, key=keyfn)

    @staticmethod
    def _make_sympy_locals(var_names):
        loc = {vn: sp.Symbol(vn, real=True) for vn in var_names}
        return loc

    @staticmethod
    def _parse_constraint_line(line: str):
        line = line.strip()
        if "<=" in line:
            left, right = line.split("<=", 1)
            return left.strip(), right.strip(), "<="
        if ">=" in line:
            left, right = line.split(">=", 1)
            return left.strip(), right.strip(), ">="
        if "=" in line:
            left, right = line.split("=", 1)
            return left.strip(), right.strip(), "="
        raise ValueError(f"Unsupported constraint format: {line}")

    @classmethod
    def from_file(cls, path: str):
        with open(path, "r", encoding="utf-8") as f:
            raw_lines = [ln.rstrip("\n") for ln in f]

        constraints_started = False
        header = []
        cons_lines = []
        for ln in raw_lines:
            if not ln.strip():
                continue
            if ln.strip().startswith("#"):
                continue
            if ln.strip().lower() == "constraints:":
                constraints_started = True
                continue
            if constraints_started:
                cons_lines.append(ln.strip())
            else:
                header.append(ln.strip())

        fields = {}
        for ln in header:
            if ":" in ln:
                k, v = cls._parse_keyvals(ln)
                fields[k.lower()] = v

        if "target_function" not in fields:
            raise ValueError("Missing 'target_function:'")
        f_str = fields["target_function"]

        expr_strs = [f_str]
        for c in cons_lines:
            left, right, op = cls._parse_constraint_line(c)
            expr_strs.append(left)
            expr_strs.append(right)

        var_names = cls._find_vars_in_exprs(expr_strs)
        if not var_names:
            raise ValueError("No variables like x1, x2, ... found.")

        loc = cls._make_sympy_locals(var_names)

        f_expr = sp.sympify(f_str, locals=loc)

        c_exprs = []
        cl = []
        cu = []
        for c in cons_lines:
            left, right, op = cls._parse_constraint_line(c)
            left_expr = sp.sympify(left, locals=loc)
            right_expr = sp.sympify(right, locals=loc)
            expr = left_expr - right_expr

            if op == "=":
                c_exprs.append(expr)
                cl.append(0.0)
                cu.append(0.0)
            elif op == "<=":
                c_exprs.append(expr)
                cl.append(-np.inf)
                cu.append(0.0)
            elif op == ">=":
                c_exprs.append(expr)
                cl.append(0.0)
                cu.append(np.inf)

        if "x" not in fields:
            raise ValueError("Missing 'x:' initial point.")
        x0 = [float(t) for t in fields["x"].split()]
        if len(x0) != len(var_names):
            raise ValueError(f"x has {len(x0)} values but detected {len(var_names)} variables: {var_names}")

        s0 = None
        if "s" in fields:
            s0 = [float(t) for t in fields["s"].split()]

        lam0 = None
        if "lambdas" in fields:
            lam0 = [float(t) for t in fields["lambdas"].split()]

        mu0 = None
        if "mu" in fields:
            mu0 = float(fields["mu"])

        eps = None
        if "epsilon" in fields:
            eps = float(fields["epsilon"])

        nu0 = None
        if "nu" in fields:
            nu0 = [float(t) for t in fields["nu"].split()]

        return cls(
            var_names=var_names,
            f_expr=f_expr,
            c_exprs=c_exprs,
            cl=cl,
            cu=cu,
            x0=x0,
            s0=s0,
            lam0=lam0,
            nu0=nu0,
            mu0=mu0,
            epsilon=eps,
        )

    def obj(self, x):
        x = np.asarray(x, dtype=float)
        return float(self._f_func(*x))

    def grad(self, x):
        x = np.asarray(x, dtype=float)
        g = self._grad_func(*x)
        return np.asarray(g, dtype=float).reshape(-1)

    def cons(self, x):
        x = np.asarray(x, dtype=float)
        if self._m == 0:
            return np.zeros(0, dtype=float)
        c = self._c_func(*x)
        return np.asarray(c, dtype=float).reshape(-1)

    def jac(self, x):
        x = np.asarray(x, dtype=float)
        if self._m == 0:
            return np.zeros((0, self._n), dtype=float)
        J = self._J_func(*x)
        return np.asarray(J, dtype=float)

    def hess(self, x, y):
        x = np.asarray(x, dtype=float)
        y = np.asarray(y, dtype=float)

        if y.size != self._m:
            raise ValueError(f"hess: y has size {y.size} but m={self._m}")

        H = np.asarray(self._Hf_func(*x), dtype=float)

        for i in range(self._m):
            yi = y[i]
            if yi == 0.0:
                continue
            Hi = np.asarray(self._Hc_funcs[i](*x), dtype=float)
            H = H + yi * Hi

        return H

## Вспомогательный класс для разделения ограничений (используется внутри решателя)

In [ ]:
class _ConstraintMapper:
    def __init__(self, prob, tol_eq=0.0):
        self.prob = prob
        self.tol_eq = tol_eq

        m = prob.m
        cl = np.asarray(prob.cl, dtype=float).copy()
        cu = np.asarray(prob.cu, dtype=float).copy()

        self.eq_idx = []
        self.ineq_upper = []
        self.ineq_lower = []
        self.free_idx = []

        for i in range(m):
            li, ui = cl[i], cu[i]
            li_f = np.isfinite(li)
            ui_f = np.isfinite(ui)

            if li_f and ui_f and abs(li - ui) <= tol_eq:
                self.eq_idx.append(i)
            else:
                has_ineq = False
                if ui_f:
                    self.ineq_upper.append((i, ui))
                    has_ineq = True
                if li_f:
                    self.ineq_lower.append((i, li))
                    has_ineq = True
                if not has_ineq:
                    self.free_idx.append(i)

        self.m_eq = len(self.eq_idx)
        self.p_ineq = len(self.ineq_upper) + len(self.ineq_lower)

        self.g_spec = []
        for (i, u) in self.ineq_upper:
            self.g_spec.append((i, +1, u))
        for (i, l) in self.ineq_lower:
            self.g_spec.append((i, -1, l))

    def split_constraints(self, x):
        c = np.asarray(self.prob.cons(x), dtype=float)
        h = np.zeros(self.m_eq, dtype=float)
        for k, i in enumerate(self.eq_idx):
            h[k] = c[i] - self.prob.cl[i]

        g = np.zeros(self.p_ineq, dtype=float)
        for j, (i, kind, bnd) in enumerate(self.g_spec):
            if kind == +1:
                g[j] = c[i] - bnd
            else:
                g[j] = bnd - c[i]
        return h, g

    def split_jacobians(self, x):
        J = np.asarray(self.prob.jac(x), dtype=float)
        n = self.prob.n

        Jh = np.zeros((self.m_eq, n), dtype=float)
        for k, i in enumerate(self.eq_idx):
            Jh[k, :] = J[i, :]

        Jg = np.zeros((self.p_ineq, n), dtype=float)
        for j, (i, kind, _) in enumerate(self.g_spec):
            if kind == +1:
                Jg[j, :] = J[i, :]
            else:
                Jg[j, :] = -J[i, :]
        return Jh, Jg

    def multipliers_to_original(self, lam, nu):
        y = np.zeros(self.prob.m, dtype=float)

        for k, i in enumerate(self.eq_idx):
            y[i] += lam[k]

        for j, (i, kind, _) in enumerate(self.g_spec):
            if kind == +1:
                y[i] += nu[j]
            else:
                y[i] += -nu[j]
        return y

In [ ]:
@dataclass(frozen=True)
class Solution:
    x: np.ndarray
    s: np.ndarray
    lam: np.ndarray
    nu: np.ndarray
    mu: float
    f: float

In [ ]:
class Logger:
    def __init__(self, verbose):
        self.verbose = verbose
        self._logger = logging.getLogger()
        self._logger.setLevel(logging.INFO)
        self._logger.propagate = False

        self._logger.handlers.clear()

        handler = logging.StreamHandler()
        formatter = logging.Formatter("%(message)s")
        handler.setFormatter(formatter)
        
        self._logger.addHandler(handler)

    def log(self, verbose, message):
        if verbose <= self.verbose:
            self._logger.info(message)

## Эвристика для выбора уменьшения шага

In [ ]:
class StepLearner:
    def __init__(self, initial_backtrack=0.5, learning_rate=0.05):
        self.learned_backtrack = initial_backtrack
        self.lr = learning_rate
        self.history = []

    def get_backtrack_factor(self):
        return self.learned_backtrack

    def update(self, success, alpha_final, reduction_ratio):
        if not success:
            self.learned_backtrack = max(0.1, self.learned_backtrack - self.lr)
        else:
            if reduction_ratio > 2.0 and alpha_final > 0.5:
                self.learned_backtrack = min(0.95, self.learned_backtrack + self.lr)
        
        self.history.append(self.learned_backtrack)

## Метод внутренней точки

In [ ]:
class InteriorPointMethod:
    def __init__(
        self,
        prob,
        mu0=1e-1,
        sigma=0.1,
        newton_tol=1e-8,
        outer_tol=1e-8,
        mu_min=1e-6,
        max_newton_iters=1000,
        max_outer_iters=50,
        alpha_backtrack=0.5,
        alpha_min=1e-10,
        positivity_eta=0.99,
        verbose=1
    ):
        if not isinstance(prob, NLPProblemInterface):
            raise TypeError("prob должен реализовывать NLPProblemInterface")
        
        self.prob = prob
        self._mapper = _ConstraintMapper(prob)
        self._logger = Logger(verbose)

        if getattr(prob, "mu0", None) is not None:
            mu0 = prob.mu0
        if getattr(prob, "epsilon", None) is not None:
            outer_tol = prob.epsilon

        self.step_learner = StepLearner(initial_backtrack=alpha_backtrack)

        self.mu0 = float(mu0)
        self.sigma = float(sigma)
        self.newton_tol = float(newton_tol)
        self.outer_tol = float(outer_tol)
        self.mu_min = float(mu_min)
        self.max_newton_iters = int(max_newton_iters)
        self.max_outer_iters = int(max_outer_iters)
        self.alpha_backtrack = float(alpha_backtrack)
        self.alpha_min = float(alpha_min)
        self.positivity_eta = float(positivity_eta)
        self.verbose = bool(verbose)

    def _F_and_J(self, x, s, lam, nu, mu):
        n = self.prob.n
        m = self._mapper.m_eq
        p = self._mapper.p_ineq

        grad_f = np.asarray(self.prob.grad(x), dtype=float)
        h, g = self._mapper.split_constraints(x)
        Jh, Jg = self._mapper.split_jacobians(x)

        r_dual = grad_f.copy()
        if m > 0:
            r_dual += Jh.T @ lam
        if p > 0:
            r_dual += Jg.T @ nu

        r_cent = s * nu - mu * np.ones(p, dtype=float)
        r_pri_eq = h.copy()
        r_pri_ineq = g + s

        F = np.concatenate([r_dual, r_cent, r_pri_eq, r_pri_ineq])

        y = self._mapper.multipliers_to_original(lam, nu)
        H = np.asarray(self.prob.hess(x, y), dtype=float)

        dim = n + p + m + p
        J = np.zeros((dim, dim), dtype=float)

        ix0, ix1 = 0, n
        is0, is1 = ix1, ix1 + p
        il0, il1 = is1, is1 + m
        inu0, inu1 = il1, il1 + p

        J[ix0:ix1, ix0:ix1] = H
        if m > 0:
            J[ix0:ix1, il0:il1] = Jh.T
        if p > 0:
            J[ix0:ix1, inu0:inu1] = Jg.T

        if p > 0:
            J[is0:is1, is0:is1] = np.diag(nu)
            J[is0:is1, inu0:inu1] = np.diag(s)

        if m > 0:
            J[il0:il1, ix0:ix1] = Jh

        if p > 0:
            J[inu0:inu1, ix0:ix1] = Jg
            J[inu0:inu1, is0:is1] = np.eye(p)

        return F, J

    def _initial_point(self, mu):
        n = self.prob.n
        x = np.asarray(self.prob.x0, dtype=float).copy()

        _, g = self._mapper.split_constraints(x)
        p = self._mapper.p_ineq

        s0 = getattr(self.prob, "s0", None)
        lam0 = getattr(self.prob, "lam0", None)
        nu0 = getattr(self.prob, "nu0", None)

        if p > 0:
            if s0 is not None and len(s0) == p:
                s = np.asarray(s0, dtype=float).copy()
                s = np.maximum(s, 1e-8)
            else:
                s = np.maximum(1.0, -g + 1.0)

            if nu0 is not None and len(nu0) == p:
                nu = np.asarray(nu0, dtype=float).copy()
                nu = np.maximum(nu, 1e-8)
            else:
                nu = mu / s
        else:
            s = np.zeros(0, dtype=float)
            nu = np.zeros(0, dtype=float)

        m = self._mapper.m_eq
        if m > 0:
            if lam0 is not None and len(lam0) == m:
                lam = np.asarray(lam0, dtype=float).copy()
            else:
                lam = np.zeros(m, dtype=float)
        else:
            lam = np.zeros(0, dtype=float)

        return x, s, lam, nu

    def solve(self):
        solve_start = perf_counter()

        mu = self.mu0
        x, s, lam, nu = self._initial_point(mu)

        n = self.prob.n
        m = self._mapper.m_eq
        p = self._mapper.p_ineq

        def normF(x_, s_, lam_, nu_, mu_):
            F_, _ = self._F_and_J(x_, s_, lam_, nu_, mu_)
            return np.linalg.norm(F_, ord=2)

        def get_norm(x_, s_, lam_, nu_, mu_):
            F_v, _ = self._F_and_J(x_, s_, lam_, nu_, mu_)
            return np.linalg.norm(F_v, ord=2)
        
        for outer_it in range(1, self.max_outer_iters + 1):
            outer_start = perf_counter()
            
            for newton_it in range(1, self.max_newton_iters + 1):
                F, J = self._F_and_J(x, s, lam, nu, mu)
                F_norm = np.linalg.norm(F, ord=2)
                
                self._logger.log(1, f"Внешняя итерация {outer_it:02d}, при µ = {mu:.3e}. Итерация Ньютона {newton_it:02d}, ||F|| = {F_norm:.3e}.")

                if F_norm <= self.newton_tol:
                    break

                try:
                    dz = np.linalg.solve(J, -F)
                except np.linalg.LinAlgError:
                    dz, *_ = np.linalg.lstsq(J, -F, rcond=None)

                dx = dz[0:n]
                ds = dz[n:n+p]
                dlam = dz[n+p:n+p+m]
                dnu = dz[n+p+m:n+p+m+p]

                alpha = 1.0

                if p > 0:
                    neg_ds = ds < 0
                    neg_dnu = dnu < 0
                    if np.any(neg_ds):
                        alpha = min(alpha, self.positivity_eta * np.min(-s[neg_ds] / ds[neg_ds]))
                    if np.any(neg_dnu):
                        alpha = min(alpha, self.positivity_eta * np.min(-nu[neg_dnu] / dnu[neg_dnu]))
                    alpha = min(alpha, 1.0)

                base_norm = F_norm
                success = False
                current_backtrack = self.step_learner.get_backtrack_factor()

                while alpha >= self.alpha_min:
                    xn = x + alpha * dx
                    sn = s + alpha * ds
                    lamn = lam + alpha * dlam
                    nun = nu + alpha * dnu

                    if p > 0 and (np.any(sn <= 0) or np.any(nun <= 0)):
                        alpha *= current_backtrack
                        continue
                    
                    new_norm = get_norm(xn, sn, lamn, nun, mu)
                    if new_norm > base_norm:
                        alpha *= current_backtrack
                        continue

                    reduction = base_norm / (new_norm + 1e-18)
                    self.step_learner.update(True, alpha, reduction)
                    x, s, lam, nu, success = xn, sn, lamn, nun, True
                    break
                
                if not success:
                    self.step_learner.update(False, alpha, 0)

                    if alpha < self.alpha_min:
                        self._logger.log(1, "Размер шага стал слишком мал; остановка метода Ньютона для этого μ.")

                    break

            outer_time = perf_counter() - outer_start

            self._logger.log(2, f"Время работы внешней итерации: {outer_time:.2f}")

            F_norm = normF(x, s, lam, nu, mu)
            if (mu <= self.mu_min) and (F_norm <= self.outer_tol):
                self._logger.log(1, "Решение сошлось.")
                break

            mu = self.sigma * mu
            if p > 0:
                nu = np.maximum(nu, 1e-16)
                s = np.maximum(s, 1e-16)

        solve_time = perf_counter() - solve_start

        self._logger.log(2, f"Время решения: {solve_time:.2f} с.")

        f_star = float(self.prob.obj(x))

        self._logger.log(1, f"Найденное решение: x* = ({', '.join([str(xi) for xi in x])}), f(x*) = {f_star}.")

        return Solution(
            x=x,
            s=s,
            lam=lam,
            nu=nu,
            mu=mu,
            f=f_star
        )

## Примеры использования

### Пример 1: Решение задачи из txt-файла

In [ ]:
txt_problem = TxtNLPProblem.from_file("input.txt")
solver = InteriorPointMethod(txt_problem, verbose=2)
solution = solver.solve()

### Пример 2: Решение задачи из pycutest

In [ ]:
!pip install meson
!export PATH="$HOME/.local/bin:$PATH"
!meson --version

In [ ]:
!sudo apt -y update
!sudo apt -y install ninja-build
!ninja --version

In [ ]:
!mkdir cutest
%cd cutest
!git clone https://github.com/ralna/SIFDecode ./sifdecode
!git clone https://github.com/ralna/CUTEst ./cutest
!git clone https://bitbucket.org/optrove/sif ./mastsif

In [ ]:
%cd sifdecode
!meson setup builddir
!meson compile -C builddir
!sudo meson install -C builddir

In [ ]:
%cd ../cutest
!meson setup builddir -Dmodules=false
!meson compile -C builddir
!sudo meson install -C builddir

In [ ]:
%env MASTSIF=/content/cutest/mastsif

In [ ]:
!pip install pycutest

In [ ]:
import pycutest

In [ ]:
cutest_problem_name = "HS21"

cutest_raw = pycutest.import_problem(cutest_problem_name)
cutest_problem = PyCUTestProblem(cutest_raw)

solver = InteriorPointMethod(cutest_problem, mu0=1e-1, sigma=0.1, verbose=2)
solution = solver.solve()